In [1]:
from numpy import *
import random, math

def loadDataSetY(fileName):
    X = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split(' ')
        fltLine = map(float, curLine)
        #print(curLine[0] + '/' + curLine[1] + '/' + curLine[3])
        X.append(float(curLine[2]))
        #print(lst[0])
        #X.append(lst)
        #print(len(curLine))
    return X
def loadDataSet(fileName, id):
    X = []
    cnt = 0
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split(' ')
        fltLine = map(float, curLine)
        cnt = cnt + 1
        #print(curLine[0] + '/' + curLine[1] + '/' + curLine[2])
        #print(cnt)
        lst = []
        lst.append(float(curLine[0]))
        lst.append(float(curLine[1]))
        if (id == 0):
            X.append(lst)
        elif (float(curLine[2]) == id):
            X.append(lst)
            #print(lst[0])
        #print(lst[0])
        #X.append(lst)
        #print(len(curLine))
    return X

In [2]:
def calc_prob(pMiu,pSigma,X,K,N,D):
    Px = mat(zeros([N, K]))
    for k in range(K):
        Xshift = X-tile(pMiu[k, :],(N, 1)) #N * D
        inv_pSigma = linalg.pinv(mat(pSigma[:, :, k])) # D * D

        tmp = sum(array((Xshift*inv_pSigma)) * array(Xshift), 1) # N * 1
        #print(tmp)
        #tmp = mat(tmp).T

        Sigema = linalg.det(mat(inv_pSigma))

        if Sigema < 0:
            Sigema=0

        coef = power((2*(math.pi)),(-D/2)) * sqrt(Sigema)              
        tmp = tmp.reshape([N, 1])
        
        Px[:, k] = coef * exp(-0.5*tmp)
        #print(Px[:, k])
    return Px

In [3]:
def init_params(X,centroids,K,N,D):  
    pMiu = centroids #k*D
    pPi = zeros([1, K])
    pSigma = zeros([D, D, K]) 
    
    distmat = tile(sum(power(X,2), 1),(1, K)) + \
        tile(transpose(sum(power(pMiu,2), 1)),(N, 1)) -  \
        2*X*transpose(pMiu)
    labels = distmat.argmin(1)

    for k in range(K):
        boolList = (labels==k).tolist()
        indexList = [boolList.index(i) for i in boolList if i==[True]]
        Xk = X[indexList, :]
        pPi[0][k] = float(size(Xk, 0))/N
        #print(Xk.shape)
    for k in range(K):
        for d in range(D):
            pSigma[d, d, k] = 1
    return pMiu,pPi,pSigma


In [4]:
from sklearn.cluster import KMeans
from sklearn.externals import joblib
import numpy
def gmm(file, K_or_centroids, op):
    threshold = 0.2#5e-3
    eps = 1e-9
    dataMat = mat(loadDataSet(file, op))
    print(dataMat)
    [N, D] = shape(dataMat)

    K = K_or_centroids
    #Rn_index = list(range(N))
    #print(Rn_index)
    #print(type(Rn_index))
    #random.shuffle(Rn_index) 
    #centroids = dataMat[Rn_index[0:K], :]; 
    clf = KMeans(n_clusters=K)
    s = clf.fit(dataMat)
    print(s)

    centroids = clf.cluster_centers_
    print(centroids)
    
    [pMiu,pPi,pSigma] = init_params(dataMat,centroids,K,N,D)      
    Lprev = -inf 
    #print('pMu:', pMiu)
    #print('pPi:', pPi)
    #print(sum(pPi, 1))
    #print('pSigma:',pSigma)
    #for kk in range(K):
    #    print(linalg.det(mat(pSigma[:, :, kk])))
    print(N, K, D)
    while True:
        Px = calc_prob(pMiu,pSigma,dataMat,K,N,D)
        #print(Px)
        pGamma = mat(array(Px) * array(tile(pPi, (N, 1))))  
        pGamma = pGamma / tile(sum(pGamma, 1), (1, K)) 
        
        Nk = sum(pGamma, 0)
    
        pMiu = mat(diag((1/Nk).tolist()[0])) * (pGamma.T) * dataMat 
        pPi = Nk/N

        #print('kk=',K)
        for kk in range(K):
            Xshift = dataMat-tile(pMiu[kk], (N, 1))  
            #print(Xshift.shape)
            #Xshift.T * mat(diag(pGamma[:, kk].T.tolist()[0])) *  Xshift / 2
            #print(mat(diag(pGamma[:, kk].T.tolist()[0])).shape)
            #print(Nk.shape)
            #print(Nk[0,kk])
            pSigma[:, :, kk] = (Xshift.T * \
                mat(diag(pGamma[:, kk].T.tolist()[0])) * Xshift) / Nk[0,kk]

        L = sum(log(Px*(pPi.T)))  
        print("add:", L-Lprev)
        if L-Lprev < threshold:
            break        
            
        Lprev = L
    return pMiu, pPi, pSigma


In [5]:
def getposs(pMiu, pPi, pSigma, X, K):
    N = X.shape[0]
    print(X.shape)
    Px = calc_prob(pMiu,pSigma,X,K,N,2)
    pGamma = mat(array(Px) * array(tile(pPi, (N, 1)))) 
    pGamma = sum(pGamma, 1) #N
    return pGamma

In [ ]:
def doit():
    dir = 'C:/Users/UsedToBe/Desktop/PyLab/GMM/newdata/train.txt'
    dir2 = 'C:/Users/UsedToBe/Desktop/PyLab/GMM/newdata/test.txt'
    X = mat(loadDataSet(dir2, 0))
    N = X.shape[0]
    print(N)
    K = 10
    t1, t2, t3 = gmm(dir, K, 1)
    A = getposs(t1, t2, t3, X, K) 
    t1, t2, t3 = gmm(dir, K, 2)
    B = getposs(t1, t2, t3, X, K)
    cnt = 0
    Y = loadDataSetY("C:/Users/UsedToBe/Desktop/PyLab/GMM/newdata/test.txt")
    for i in range(N):
        if A[i] > B[i]:
            cnt += (Y[i] == 1)
        else:
            cnt += (Y[i] == 2)
    print("True = %d" % cnt)
    with open("C:/Users/UsedToBe/Desktop/PyLab/GMM/res.txt", 'w') as f:
        for i in range(N):
            if A[i] > B[i]:
                f.write('1\n')
            else:
                f.write('2\n')
    if cnt > 786:
        return True
    else:
        return False

In [ ]:
while 1:
    f = doit()
    if f == True:
        break

800
[[ 2.982154  0.275734]
 [ 1.216709 -0.804729]
 [ 3.077832  0.307613]
 ..., 
 [ 0.733904  0.143319]
 [ 2.9561    0.047025]
 [ 3.197617  0.046594]]
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
[[ 2.9710914   0.24679221]
 [ 0.41019709  0.64758753]
 [ 1.78127954  2.1323385 ]
 [ 0.99333148 -0.85565684]
 [ 3.02394881 -0.25397272]
 [ 2.20572246  2.10352422]
 [ 1.99532445  1.76875669]
 [ 0.67608909 -1.11888034]
 [ 0.57641897  0.31942761]
 [ 1.35164411 -1.08307944]]
2800 10 2
add: inf
add: 1438.82370813
add: 1775.13827598
add: 744.271587625
add: 366.970203264
add: 123.504710655
add: 32.6812170602
add: 15.74579077
add: 7.05748372233
add: 2.90033982355
add: 1.4246251602
add: 0.850917156365
add: 0.58355232366
add: 0.436550798388
add: 0.345091934149
add: 0.283489808947
add: 0.240129371373
add: 0.208963683762
add: 0.18637391548
(800, 2)
[[ 1.124586  1.491173

add: 0.17912741625
(800, 2)
[[ 1.124586  1.491173]
 [-0.367243  0.068235]
 [ 1.165453  1.627965]
 ..., 
 [-0.372653  0.03516 ]
 [ 2.509096  1.295253]
 [-0.603547  0.46768 ]]
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
[[-0.2436045  -0.00915523]
 [ 1.31697951  0.19094971]
 [ 1.16923969  1.5812935 ]
 [ 2.75569812  1.0107945 ]
 [ 2.28574507  0.9986793 ]
 [ 1.35576484 -0.21001799]
 [-0.62815574 -0.1833755 ]
 [ 0.8363523   1.43080549]
 [-0.57507829  0.21866767]
 [ 1.80596777  0.01496122]]
2800 10 2
add: inf
add: 1430.1391748
add: 656.228417171
add: 607.545471015
add: 583.694020689
add: 478.383425136
add: 195.018075853
add: 69.7819522602
add: 22.0715883324
add: 6.96388135734
add: 3.09603908379
add: 1.96476609556
add: 1.43025088988
add: 1.10612710963
add: 0.886677781356
add: 0.727373006255
add: 0.606513454291
add: 0.513116135226
add: 0.441224265644
add: 

add: 24.6472980569
add: 8.8480676967
add: 4.24326349094
add: 2.78454557043
add: 2.04145180458
add: 1.5655539017
add: 1.23411753969
add: 0.993840641146
add: 0.817196889818
add: 0.687425706477
add: 0.591525829244
add: 0.517864004107
add: 0.457256271862
add: 0.405581740205
add: 0.364029725488
add: 0.335378681971
add: 0.320091238261
add: 0.315505074936
add: 0.317193635935
add: 0.320655328233
add: 0.322594623284
add: 0.321483250324
add: 0.317221566794
add: 0.31028201235
add: 0.301049461384
add: 0.289670768503
add: 0.27621374904
add: 0.260838702993
add: 0.24386216136
add: 0.225754939751
add: 0.207136410307
add: 0.188754328063
(800, 2)
True = 785
800
[[ 2.982154  0.275734]
 [ 1.216709 -0.804729]
 [ 3.077832  0.307613]
 ..., 
 [ 0.733904  0.143319]
 [ 2.9561    0.047025]
 [ 3.197617  0.046594]]
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
[[ 2.97153928  0.

add: inf
add: 1454.27062068
add: 1786.05645436
add: 668.333689399
add: 414.701822812
add: 138.031198154
add: 32.9911464453
add: 7.73573008912
add: 3.979860975
add: 2.66007717207
add: 1.77086411925
add: 1.14402084483
add: 0.738331892228
add: 0.496819136642
add: 0.355551453553
add: 0.27025123891
add: 0.216305288952
add: 0.180745819031
(800, 2)
[[ 1.124586  1.491173]
 [-0.367243  0.068235]
 [ 1.165453  1.627965]
 ..., 
 [-0.372653  0.03516 ]
 [ 2.509096  1.295253]
 [-0.603547  0.46768 ]]
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
[[ 0.81919011  1.47964583]
 [-0.30988431 -0.17383006]
 [ 1.80596777  0.01496122]
 [ 2.75569812  1.0107945 ]
 [-0.38613133  0.2250967 ]
 [ 1.31666019  0.19185463]
 [ 1.18106286  1.53190896]
 [-0.70582806 -0.02282749]
 [ 1.35589724 -0.20911665]
 [ 2.28753771  0.99835755]]
2800 10 2
add: inf
add: 1432.01900335
add: 660.3822608

add: inf
add: 1430.22894069
add: 658.592011045
add: 613.823010864
add: 584.481842111
add: 471.978565755
add: 188.758467616
add: 71.5851830117
add: 23.40134069
add: 7.23764074287
add: 3.20301378134
add: 2.01564502077
add: 1.4589107236
add: 1.12597345685
add: 0.903197559527
add: 0.742854362845
add: 0.62187619508
add: 0.528679127413
add: 0.457053695713
add: 0.402989178653
add: 0.363232415489
add: 0.334867281156
add: 0.315320600399
add: 0.302424744508
add: 0.294403507957
add: 0.289786886369
add: 0.287306512696
add: 0.285819376043
add: 0.28428265609
add: 0.28177436236
add: 0.277537710243
add: 0.271029396304
add: 0.261963674281
add: 0.250346544654
add: 0.23648372435
add: 0.22094111687
add: 0.204452918629
add: 0.187799685267
(800, 2)
True = 784
800
[[ 2.982154  0.275734]
 [ 1.216709 -0.804729]
 [ 3.077832  0.307613]
 ..., 
 [ 0.733904  0.143319]
 [ 2.9561    0.047025]
 [ 3.197617  0.046594]]
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_